<a href="https://colab.research.google.com/github/sudarshan-koirala/youtube-stuffs/blob/main/langchain/Chat_with_Any_Documents_Own_ChatGPT_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Environment Setup

In [1]:
import langchain
print(langchain.__version__)


0.0.179


In [2]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base =  os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("AZ_OPENAI_API_KEY")

# Initialize gpt-35-turbo and our embedding model
llm = AzureChatOpenAI(deployment_name="cresen-gpt-35-turbo", openai_api_version="2023-03-15-preview") # For Chat
embeddings = OpenAIEmbeddings(model = 'text-embedding-ada-002', deployment='text-embedding-ada-002',chunk_size=1) # For Embeddings
#embeddings = OpenAIEmbeddings(deployment="cresen-embedding",model="text-embedding-ada-002")

## 2. Documents Loading

In [3]:
from langchain.document_loaders import DirectoryLoader
pdf_loader = DirectoryLoader(r"C:\Users\Ananthu Raj C M\Documents\Python\ChatGPT_Trials\ChatGPT-Tabular-Data-main\Files", glob="**/*.pdf")
#take all the loader
loaders = [pdf_loader] # , readme_loader, txt_loader
#lets create document 
documents = []
for loader in loaders:
    documents.extend(loader.load())
print (f'You have {len(documents)} document(s) in your data')
print (f'There are a total of {len(documents[0].page_content)} characters in your documents')

You have 3 document(s) in your data
There are a total of 7624 characters in your documents


## 3. Splitting Text from the documents and creating Chunks

In [4]:
from langchain.text_splitter import TokenTextSplitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=20) #chunk overlap seems to work better
#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=40) 
documents = text_splitter.split_documents(documents)
print(len(documents))

6


In [5]:
documents[0]

Document(page_content="666 Exton Commons Exton PA 19341 (484) 879-426\n\nTaking the mystery and risk out of your Healthcare Engagements with EngageMate by Cresen Solutions\n\nCresen Solutions was formed to provide distinct and qualified services to the life sciences industry.\n\nOur organization has a clear focus and mission to enable improved business outcomes through advanced technology solutions and analytical expertise. The leadership team is comprised of seasoned healthcare industry experts focused on enabling the technological advancements necessary in today's ever-changing healthcare landscape. Strong foundations in technology systems and solutions and the business acumen to enable predictable business results are keys to our success. Experience, quality, and innovation are the cornerstones of our team's capabilities and are applied to each engagement.\n\nEngageMate© is an HCP/HCO engagements tool developed by Cresen Solutions that enables our\n\nclients to fully track HCP and H

## 4. Embedding with Azure Embedding Model and storing it in ChromaDB Vectorestore

Langchain and ChromaDB : https://blog.langchain.dev/langchain-chroma/

In [6]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
from langchain.vectorstores import Chroma
persist_directory = 'db'
vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory=persist_directory)
vectorstore.persist()
vectorstore = None

Using embedded DuckDB with persistence: data will be stored in: db


In [ ]:
#from langchain.vectorstores import FAISS
#db = FAISS.from_documents(documents=documents, embedding=embeddings)

In [7]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":2})
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), retriever)

AttributeError: 'NoneType' object has no attribute 'as_retriever'

In [19]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=vectorstore.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)


In [20]:
chat_history = []
query = "what is Spendmate?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])


Question: what is Spendmate?
Answer: SpendMate is a healthcare compliance transparency tool designed to streamline the end-to-end business process of transparency reporting. It enables companies to meet the growing demand for transparency between industry and medical providers in the form of report preparation, generation, and submission, doing so cost-effectively, and it also easily scales with companies as they expand globally. SpendMate simplifies the spend reporting process by providing the flexibility and scalability needed by clients to meet the compliance demands of external entities.


In [21]:
chat_history = [(query, result["answer"])]
query = "Which products does cresen solutions have?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])


Question: Which products does cresen solutions have?
Answer: Cresen Solutions offers multiple products including EngageMate, MonitorMate, and SpendMate. They also offer a free PowerCMS tool for analyzing Open Payments data.


## Setup

%%capture
!pip install openai langchain  tiktoken pypdf unstructured[local-inference] gradio chromadb
!pip install watermark
!pip install chromadb
pip install unstructured
pip install pdf2image
pip install pytesseract
pip install detectron2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [ ]:
%reload_ext watermark
%watermark -a "Ananthu Raj C M" -vmp langchain,openai,chromadb

In [8]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

from langchain.chat_models import AzureChatOpenAI


### Using Chroma for storing vectors

In [ ]:
pip install chromadb

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
from langchain.vectorstores import Chroma
persist_directory = 'db'

embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory=persist_directory)

In [ ]:
vectorstore.persist()
vectorstore = None

## Querying from ChromaDB

In [11]:
vectorstore = Chroma.from_documents(documents, embeddings,persist_directory=persist_directory)

Using embedded DuckDB with persistence: data will be stored in: db


In [12]:
# Now we can load the persisted database from disk, and use it as normal. 
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

Using embedded DuckDB with persistence: data will be stored in: db


In [13]:
query = "What is spendmate ?"
docs = vectorstore.similarity_search(query)

In [ ]:
len(docs) #it went on and search on the 4 different vectors to find the similarity

In [ ]:
print(docs[0].page_content)

In [ ]:
print(docs[1].page_content)

## Now the langchain part (Chaining with Chat History) 
- There are many chains but we use this [link](https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html)

In [ ]:
from langchain.llms import OpenAI

In [16]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":2})
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever)

In [ ]:
chat_history = []
query = "How is spendmate?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

In [ ]:
chat_history.append((query, result["answer"]))
chat_history

In [ ]:
query = "What is this number multiplied by 2?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

## Create a chatbot with memory with  widgets

In [17]:
from IPython.display import display
import ipywidgets as widgets

In [18]:
chat_history = []

def on_submit(_):
    query = input_box.value
    input_box.value = ""
    
    if query.lower() == 'exit':
        print("Thanks for the chat!")
        return
    
    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))
    
    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="Orange">Chatbot:</font></b> {result["answer"]}'))

print("Chat with your data. Type 'exit' to stop")

input_box = widgets.Text(placeholder='Please enter your question:')
input_box.on_submit(on_submit)

display(input_box)

Chat with your data. Type 'exit' to stop


Text(value='', placeholder='Please enter your question:')

HTML(value='<b>User:</b> What is spendmate ?')

HTML(value='<b><font color="Orange">Chatbot:</font></b> SpendMate is a healthcare compliance transparency tool…

HTML(value='<b>User:</b> What are the products available at cresen solutions ?')

HTML(value='<b><font color="Orange">Chatbot:</font></b> Cresen Solutions offers three flagship products: Engag…

## Gradio Part (Building the [chatbot like UI](https://gradio.app/docs/#chatbot))

### Gradio sample example

In [ ]:
import gradio as gr
import random

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        print(message)
        print(chat_history)
        bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
        chat_history.append((message, bot_message))
        print(chat_history)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True, share=True)

### Gradio langchain example

In [ ]:
import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    
    def respond(user_message, chat_history):
        print(user_message)
        print(chat_history)
        # Get response from QA chain
        response = qa({"question": user_message, "chat_history": chat_history})
        # Append user message and response to chat history
        chat_history.append((user_message, response["answer"]))
        print(chat_history)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True, share=True)